Index-Building:
The factors used include the following:
- planet mass (Mjup)
- planet radius (Rjup)
- planet temperature (K)
- planet density CALCULATE
- radiation intensity (W/m^2) CALCULATE
- star distance (pc)
- AND/OR HZ measurement CALCULATE
- star metallicity ?? fraction
- star mass (Msun)
- sun radius (Rsun)
- star age ?? Gy
- star effective temperature (K)

The index will consist of the following:
- planet mass +
planet radius +
planet temperature +
star distance (HZ) + 
radiation intensity

Weighing can be determined in 1 of 2 ways:
- use a correlation regression as done in Cobb Douglas
- use difference from "ideal" Earth on a normal distribution to determine the importance of outliers

1. Import data and packages

In [2]:
# import packages
import numpy as np
import pandas as pd
import math 
from math import pi


# import data
data = pd.read_csv('C:/Users/21sko/Desktop/dissertation/data/exo_data.csv')

#display data
# print(data)

2. Calculate density

2.1 Convert: 
- M(jup) to M(earth)
- R(jup) to R(earth)

In [6]:
# M(jup) to M(earth)
def mjup_to_mearth(planet_mass_mjup):
    mass_mearth = planet_mass_mjup * 317.8 

    return mass_mearth

data['planet_mass_mearth'] = data.apply(lambda row: mjup_to_mearth(row['planet_mass_mjup']), axis = 1)



# R(jup) to R(earth)
def rjup_to_rearth(planet_radius_rjup):
    radius_rearth = planet_radius_rjup * 11.2

    return radius_rearth

data['planet_radius_rearth'] = data.apply(lambda row: rjup_to_rearth(row['planet_radius_rjup']), axis = 1)

print(data)

         planet  planet_mass_mjup  planet_radius_rjup  temp_calculated_K  \
0  COCONUTS-2 b          6.400000            1.120000             434.00   
1    HAT-P-18 b          0.183000            0.947000             841.00   
2    HAT-P-42 b          0.975000            1.277000            1427.00   
3   kappa And b         13.000000            1.200000            1850.00   
4  TRAPPIST-1 b          0.002700            0.096890             451.55   
5    WASP-189 b          1.990000            1.619000            3394.00   
6         Earth          0.003146            0.089286             288.00   

     star_name  star_distance_pc  star_metallicity  star_mass_msun  \
0  COCONUTS-2A         10.890000            0.0000           0.370   
1     HAT-P-18        166.000000            0.1000           0.770   
2     HAT-P-42        447.000000            0.2700           1.179   
3    kappa And         50.000000           -0.3600           2.800   
4   TRAPPIST-1         12.100000         

2.2 Calculate volume 

In [10]:
# 4/3 * pi* r^3
def volume_formula(planet_radius_rearth):
    volume = (4/3) * np.pi * (planet_radius_rearth ** 3)

    return volume

data['planet_volume_rearth3'] = data.apply(lambda row: volume_formula(row['planet_radius_rearth']), axis = 1)

# print(data)
print(data)

         planet  planet_mass_mjup  planet_radius_rjup  temp_calculated_K  \
0  COCONUTS-2 b          6.400000            1.120000             434.00   
1    HAT-P-18 b          0.183000            0.947000             841.00   
2    HAT-P-42 b          0.975000            1.277000            1427.00   
3   kappa And b         13.000000            1.200000            1850.00   
4  TRAPPIST-1 b          0.002700            0.096890             451.55   
5    WASP-189 b          1.990000            1.619000            3394.00   
6         Earth          0.003146            0.089286             288.00   

     star_name  star_distance_pc  star_metallicity  star_mass_msun  \
0  COCONUTS-2A         10.890000            0.0000           0.370   
1     HAT-P-18        166.000000            0.1000           0.770   
2     HAT-P-42        447.000000            0.2700           1.179   
3    kappa And         50.000000           -0.3600           2.800   
4   TRAPPIST-1         12.100000         

2.3 Calculate relative density

In [11]:
# D = M / V

def density_formula(planet_mass_mearth, planet_volume_rearth):
    density = (planet_mass_mearth) / (planet_volume_rearth)
    
    return density


data['planet_density_relative'] = data.apply(lambda row: density_formula(row['planet_mass_mearth'], row['planet_radius_rearth']), axis = 1)

print(data)

         planet  planet_mass_mjup  planet_radius_rjup  temp_calculated_K  \
0  COCONUTS-2 b          6.400000            1.120000             434.00   
1    HAT-P-18 b          0.183000            0.947000             841.00   
2    HAT-P-42 b          0.975000            1.277000            1427.00   
3   kappa And b         13.000000            1.200000            1850.00   
4  TRAPPIST-1 b          0.002700            0.096890             451.55   
5    WASP-189 b          1.990000            1.619000            3394.00   
6         Earth          0.003146            0.089286             288.00   

     star_name  star_distance_pc  star_metallicity  star_mass_msun  \
0  COCONUTS-2A         10.890000            0.0000           0.370   
1     HAT-P-18        166.000000            0.1000           0.770   
2     HAT-P-42        447.000000            0.2700           1.179   
3    kappa And         50.000000           -0.3600           2.800   
4   TRAPPIST-1         12.100000         

3. Calculate Habitability Zone boundaries

3.1 Version 1: simpler

In [15]:
# Function to convert mass to luminosity 
def mass_to_luminosity(star_mass_msun):
    return star_mass_msun  ** 3.6

# Function to calculate habitable zone boundaries 
def calculate_habitable_zone(luminosity):
    inner_boundary = np.sqrt(luminosity / 1.1)
    outer_boundary = np.sqrt(luminosity / 0.53)
    return inner_boundary, outer_boundary 

# Convert mass to luminosity 
data['Luminosity_Lsun'] = data['star_mass_msun'].apply(mass_to_luminosity) 

# Calculate HZ boundaries and add them to the dataframe 
data[['HZ_inner_AU_s', 'HZ_outer_AU_s']] = data['Luminosity_Lsun'].apply( lambda luminosity: calculate_habitable_zone(luminosity) ).apply(pd.Series)

# Display the updated dataset to the user in a Jupyter-friendly format 
print(data)

         planet  planet_mass_mjup  planet_radius_rjup  temp_calculated_K  \
0  COCONUTS-2 b          6.400000            1.120000             434.00   
1    HAT-P-18 b          0.183000            0.947000             841.00   
2    HAT-P-42 b          0.975000            1.277000            1427.00   
3   kappa And b         13.000000            1.200000            1850.00   
4  TRAPPIST-1 b          0.002700            0.096890             451.55   
5    WASP-189 b          1.990000            1.619000            3394.00   
6         Earth          0.003146            0.089286             288.00   

     star_name  star_distance_pc  star_metallicity  star_mass_msun  \
0  COCONUTS-2A         10.890000            0.0000           0.370   
1     HAT-P-18        166.000000            0.1000           0.770   
2     HAT-P-42        447.000000            0.2700           1.179   
3    kappa And         50.000000           -0.3600           2.800   
4   TRAPPIST-1         12.100000         

3.2 Version 2: More complex

In [17]:
# inner flux 
def s_inner(Teff):
    s_inner = 1.296 - 2.139e10**-4 * Teff + 4.19e10**-8 * (Teff)**2

    return s_inner

data['S_inner'] = data['star_teff_K'].apply(s_inner) 

# outer flux
def s_outer(Teff):
    s_outer = 0.234 - 1.319e10**-5 * Teff + 6.19e10**-10 * (Teff)**2

    return s_outer

data['S_outer'] = data['star_teff_K'].apply(s_outer) 

# inner boundary
def r_inner(Luminosity, S_inner):
    r_inner = np.sqrt(Luminosity / S_inner)

    return r_inner

data['R_inner'] = data.apply(lambda row: r_inner(row['Luminosity_Lsun'], row['S_inner']), axis = 1)


# outer boundary
def r_outer(Luminosity, S_outer):
    r_outer = np.sqrt(Luminosity / S_outer)

    return r_outer

data['R_outer'] = data.apply(lambda row: r_outer(row['Luminosity_Lsun'], row['S_outer']), axis = 1)

print(data)

         planet  planet_mass_mjup  planet_radius_rjup  temp_calculated_K  \
0  COCONUTS-2 b          6.400000            1.120000             434.00   
1    HAT-P-18 b          0.183000            0.947000             841.00   
2    HAT-P-42 b          0.975000            1.277000            1427.00   
3   kappa And b         13.000000            1.200000            1850.00   
4  TRAPPIST-1 b          0.002700            0.096890             451.55   
5    WASP-189 b          1.990000            1.619000            3394.00   
6         Earth          0.003146            0.089286             288.00   

     star_name  star_distance_pc  star_metallicity  star_mass_msun  \
0  COCONUTS-2A         10.890000            0.0000           0.370   
1     HAT-P-18        166.000000            0.1000           0.770   
2     HAT-P-42        447.000000            0.2700           1.179   
3    kappa And         50.000000           -0.3600           2.800   
4   TRAPPIST-1         12.100000         